# Homography Computation for Image Rectification
# =============================================
# This script computes the homography matrix between two sets of corresponding points
# (source and destination) using both homogeneous and non-homogeneous Direct Linear
# Transformation (DLT) methods. The homography maps points from a distorted image to
# a rectified image, correcting perspective distortion.

In [2]:
import numpy as np

# Step 1: Define Input Points
# --------------------------

In [3]:
# Source points (pixel coordinates in the distorted image)
src_pts = np.array([
    [66.111, 658.33],
    [70.556, 298.33],
    [806.11, 150.56],
    [813.89, 689.44]
], dtype=np.float32)

# Destination points (pixel coordinates in the rectified image)
dst_pts = np.array([
    [0, 1000],
    [0, 0],
    [1000, 0],
    [1000, 1000]
], dtype=np.float32)

# Step 2: Homogeneous DLT
# -----------------------

In [4]:
def compute_homography_homogeneous(src_pts, dst_pts):
    """
    Compute the homography matrix using the homogeneous DLT method.
    
    Parameters:
    src_pts (numpy.ndarray): Source points [x, y] (Nx2).
    dst_pts (numpy.ndarray): Destination points [x', y'] (Nx2).
    
    Returns:
    numpy.ndarray: 3x3 homography matrix.
    """
    A = []
    for i in range(len(src_pts)):
        x, y = src_pts[i]
        x_p, y_p = dst_pts[i]
        # Two equations per point correspondence
        A.append([-x, -y, -1, 0, 0, 0, x * x_p, y * x_p, x_p])
        A.append([0, 0, 0, -x, -y, -1, x * y_p, y * y_p, y_p])
    
    A = np.array(A)
    
    # Solve using SVD
    _, _, Vt = np.linalg.svd(A)
    H = Vt[-1].reshape(3, 3)
    
    # Normalize so h33 = 1
    return H / H[2, 2]


# Step 3: Non-Homogeneous DLT
# ---------------------------

In [5]:
def compute_homography_nonhomogeneous(src_pts, dst_pts):
    """
    Compute the homography matrix using the non-homogeneous DLT method.
    
    Parameters:
    src_pts (numpy.ndarray): Source points [x, y] (Nx2).
    dst_pts (numpy.ndarray): Destination points [x', y'] (Nx2).
    
    Returns:
    numpy.ndarray: 3x3 homography matrix.
    """
    A = []
    b = []
    for i in range(len(src_pts)):
        x, y = src_pts[i]
        x_p, y_p = dst_pts[i]
        # Two equations per point correspondence
        A.append([x, y, 1, 0, 0, 0, -x * x_p, -y * x_p])
        A.append([0, 0, 0, x, y, 1, -x * y_p, -y * y_p])
        b.append(x_p)
        b.append(y_p)
    
    A = np.array(A)
    b = np.array(b)
    
    # Solve using least squares
    x = np.linalg.lstsq(A, b, rcond=None)[0]
    
    # Form the homography matrix
    H = np.array([
        [x[0], x[1], x[2]],
        [x[3], x[4], x[5]],
        [x[6], x[7], 1]
    ])
    
    return H

# Step 4: Compute Homography Matrices
# ----------------------------------

In [6]:
H_hom = compute_homography_homogeneous(src_pts, dst_pts)
H_nonhom = compute_homography_nonhomogeneous(src_pts, dst_pts)

# Step 5: Validate Homography
# --------------------------

In [7]:
def apply_homography(H, points):
    """
    Apply homography to a set of points and return Cartesian coordinates.
    
    Parameters:
    H (numpy.ndarray): 3x3 homography matrix.
    points (numpy.ndarray): Points [x, y] (Nx2).
    
    Returns:
    numpy.ndarray: Transformed points [x', y'] (Nx2).
    """
    # Convert to homogeneous coordinates
    points_hom = np.hstack([points, np.ones((points.shape[0], 1))])
    # Apply homography
    points_transformed = points_hom @ H.T
    # Normalize by w
    points_transformed /= points_transformed[:, 2:3]
    return points_transformed[:, :2]

# Transform source points
transformed_hom = apply_homography(H_hom, src_pts)
transformed_nonhom = apply_homography(H_nonhom, src_pts)

# Compute reprojection errors
errors_hom = np.sqrt(np.sum((transformed_hom - dst_pts) ** 2, axis=1))
errors_nonhom = np.sqrt(np.sum((transformed_nonhom - dst_pts) ** 2, axis=1))

# Step 6: Display Results
# -----------------------

In [8]:
print("Homography Matrix (Homogeneous DLT):")
print(np.round(H_hom, 6))
print("\nReprojection Errors (Homogeneous DLT):")
for i, error in enumerate(errors_hom):
    print(f"Point {i+1}: {error:.6f} pixels")

print("\nHomography Matrix (Non-Homogeneous DLT):")
print(np.round(H_nonhom, 6))
print("\nReprojection Errors (Non-Homogeneous DLT):")
for i, error in enumerate(errors_nonhom):
    print(f"Point {i+1}: {error:.6f} pixels")

Homography Matrix (Homogeneous DLT):
[[ 2.17133600e+00  2.68100000e-02 -1.61198992e+02]
 [ 6.03872000e-01  3.00589200e+00 -9.39354671e+02]
 [ 7.27000000e-04  4.80000000e-05  1.00000000e+00]]

Reprojection Errors (Homogeneous DLT):
Point 1: 0.000001 pixels
Point 2: 0.000000 pixels
Point 3: 0.000007 pixels
Point 4: 0.000009 pixels

Homography Matrix (Non-Homogeneous DLT):
[[ 2.17133600e+00  2.68100000e-02 -1.61198992e+02]
 [ 6.03872000e-01  3.00589200e+00 -9.39354671e+02]
 [ 7.27000000e-04  4.80000000e-05  1.00000000e+00]]

Reprojection Errors (Non-Homogeneous DLT):
Point 1: 0.000001 pixels
Point 2: 0.000000 pixels
Point 3: 0.000007 pixels
Point 4: 0.000009 pixels
